# Lab 6: The Load Wizard 및 Snowflake Marketplace

👉 이 수업에서는 파일을 로드하는 방법과 이 과정을 더 쉽게 만들어주는 Snowflake 오브젝트 유형에 대해 배웁니다. 또한 데이터와 데이터 제품을 위한 앱 스토어와 같은 역할을 하는 Snowflake Marketplace에 대해 익숙해질 것입니다.

먼저 이 실습 전반에 걸쳐 사용할 **컨텍스트 정보**를 가져오갰습니다. 

- **Start** 버튼을 클릭하여 이 노트북을 활성화합니다.

- 아래 Python 셀을 실행합니다.

#### :warning: 이 노트북에서 새 세션이 시작될 때마다, 이후 셀에서 사용할 "변수"를 설정하려면 아레 셀을 다시 실행해야 합니다. :warning:

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()
user = session.get_current_user().strip('"')
your_db = user + '_DB'
print('Your current CONTEXT information:')
print('---------------------------------')
print(session)
print('Your current USER is ' + user)

## Insert 구문은 번거롭습니다. 📓 

### Vegetable Details 테이블 데이터

팀의 비즈니스 분석가가 Rooting Depth 컬럼을 한글자로 줄이고 이를 Comma Separated Value(CSV) 파일로 출력하기로 했다고 가정해 봅시다. 이러한 변경은 겉으로 보기엔 괜찮아 보이지만, 문제가 생길 수 있습니다...
- 문제점이 무엇인지 알 수 있나요? 
- 왜 이 CSV를 로드할 수 없을까요? 
- 대부분의 행에는 몇 개의 쉼표가 있나요? 
- Hot Peppers 행에는 몇 개의 쉼표가 있나요? 

![Vegetable details table data (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_veg_details_table_1.png)

### Vegetable Details 테이블 생성 🥋 

노트북 셀에서 작업할 때, `USE` 구문을 사용하여 명시적으로 컨텍스트를 지정할 수 있다는 점을 기억하세요. 예: `USE SCHEMA db_name/schema_name`

대안으로 **전체 오브젝트 표기**를 사용할 수도 있습니다. 이는 SQL에서 오브젝트를 참조할 때 마다 오브젝트 전체 경로(또는 계층)을 사용하는 방식입니다. 이 방법은 더 길고 상세하지만, 명확성을 높이는 데 도움이 됩니다. 이 명명 규칙의 일반적인 구조는 다음과 같은 형식을 따릅니다: `<database_name>.<schema_name>.<object_name>`.

💡 **힌트**: 만약 테이블을 잘못된 데이터베이스나 스키마에 생성했다면 `ALTER TABLE - RENAME` 구문을 사용하여 이동하거나, 테이블을 삭제하고 드롭다운 메뉴의 컨텍스트 설정을 업데이트한 후 다시 생성할 수 있습니다.

In [ ]:
-- Create a Vegetable Details Table 

CREATE TABLE IF NOT EXISTS {{user}}_garden_plants.veggies.vegetable_details (
    plant_name VARCHAR(25),
    root_depth_code VARCHAR(1)
) 
DATA_RETENTION_TIME_IN_DAYS = 7;

## 파일에서 테이블 행을 로드 

### 데이터 소스

`veggie_details_a_to_k_comma_opt_enclosed.csv` 와 `veggie_details_k_to_z_pipe.csv`라는 두 개의 CSV(comma separated values) 파일이 **COMMON_DB.RESOURCES.CLASS_FILES** 스테이지에 업로드되어 있습니다.이 데이터를 이전에 생성한 **VEGETABLE_DETAILS** 테이블에 로드하려고 합니다. 

### 첫 번째 파일 다운로드 🥋 

다음 Python 코드 셀을 실행하고 생성된 링크를 클릭하여 첫 번째 파일(Artichoke to Kale)을 다운로드하세요.

In [ ]:
import streamlit as st
snowpark_df = session.sql("SELECT GET_PRESIGNED_URL(@common_db.resources.course_files, 'veggie_details_a_to_k_comma_opt_enclosed.csv')")
collected_data = snowpark_df.collect()
st.write('Click the following link to download the file:')
st.write(collected_data[0][0])

### 데이터 로드 대화 상자 단계 🥋 

1. Snowsight 오브젝트 브라우저에서 **VEGETABLE_DETAILS** 테이블을 찾습니다.
1. **Open Table details in new tab** 아이콘을 클릭해서 이 화면을 새로운 탭에 엽니다.

![Load Data dialog (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_open_table_details_1.png)

1. 오른쪽 상단 파란색 `Load Data` 버튼을 클릭합니다.
1. 다음 대화 상자가 나타납니다.
1. 웨어하우스 사용하라는 메시지가 표시되면 **(animal)_WH**로 이름 지어진 자신에게 할당되어 있는 웨어하우스를 선택합니다.

![Load Data dialog (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_load_data_1.png)

1. 파란색 **Browse** 버튼을 클릭합니다.
1. 로컬에 다운로드 받았던 `veggie_details_a_to_k_comma_opt_enclosed.csv` 이름으로 되어있는 파일을 찾고 이 파일을 열기를 선택합니다.
1. 대화 상자 오른쪽 하단 모서리에 있는 파란색 **Next** 버튼을 클릭합니다.
1. 확장된 데이터 로드 대화 상자가 나타납니다.
1. 파일에 대해 선택된 많은 기본값을 허용하지만, 왼쪽에 있는 **File format** 상자에 있는 **View options** 드롭다운 화살표를 클릭하여 일부 옵션을 조정합니다.

![Extended Load Data dialog (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_load_data_2.png)

1. **Header**에는 "Skip first line"를 선택합니다.
1. **Field optionally enclosed by**에는 "Double quotes (default)"를 선택합니다.
1. 대화 상자 오른쪽 하단 모서리에 있는 파란색 **Load** 버튼을 클릭합니다.

![Load Data file options (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_load_data_3.png)

1. 모두 정상적으로 진행되면 다음 성공 대화 상자가 보여야 합니다.
1. **Done**를 클릭하고 나타나는 페이지의 **Data Preview** 탭을 사용하여 테이블에 로드한 데이터를 검토합니다.

![Load Data success (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_load_data_success_1.png)

### 데이터 로드 시 실수에서 복구하기 Recovering from mistakes when loading data 📓

실수로 동일한 파일을 2번 로드해서 다시 시작하고자 할 경우, `TRUNCATE` 명령어를 실행하여 테이블을 비웁니다. 

`TRUNCATE TABLE (animal)_GARDEN_PLANTS.VEGGIES.VEGETABLE_DETAILS;`

그런 다음 로드 절차를 다시 시작합니다. 

## 파일에서 테이블 행 로드

### 두 번째 파일 다운로드 🥋 

다음 Python 코드 셀을 실행하고 생성된 링크를 클릭하여 두 번째 파일(Kohlrabi to Zucchini)을 다운로드하세요.

In [ ]:
snowpark_df = session.sql("SELECT GET_PRESIGNED_URL(@common_db.resources.course_files, 'veggie_details_k_to_z_pipe.csv')")
collected_data = snowpark_df.collect()
st.write('Click the following link to download the file:')
st.write(collected_data[0][0])

## 챌린지 실습: 테이블에 파일을 로드 🎯 

이전에 사용했던 동일한 **데이터 로드** 방법을 사용해서 동일한 테이블에 다운로드한 파일을 로드합니다.

💡 **참고**: **View options**에서 이 파일을 로드할 때 최소 하나 이상의 설정이 첫 번째 설정과 다를 것입니다. 이번에는 컬럼이 쉼표로 구분되지 **않습니다**. 많은 경우에 Snowflake는 자동으로 로드할 파일의 형태를 감지하고 올바른 로드를 위한 옵션을 내놓습니다. 두 번째 파일을 로드하는 경우에도 그렇나요?

![Vegetable details table data (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_veg_details_csv_1.png)

💡 **팁**: 컬럼과 행을 구분하기 위해 어떤 문자를 사용했는지 확인하기 위해 CSV 파일을 Excel로 열지 마세요. 간단한 텍스트 편집기를 사용하세요. Windows에서는 메모장이 Mac에서는 TextEdit이 잘 작동합니다.

파일을 로드한 후, `SELECT` 구문을 실행하여 테이블을 확인하고 출력 결과를 검토하세요. 42개의 행이 표시되어야 합니다. 정렬 순서를 반대로 바꾸려면 출력 테이블에서 **PLANT_NAME** 컬럼 헤더를 클릭하면 됩니다. 이렇게 하면 Zucchini가 맨 위에 나타나며 두 번째 파일이 제대로 로드되었음을 확인할 수 있습니다.

In [ ]:
SELECT * 
FROM {{user}}_garden_plants.veggies.vegetable_details;

## 테이블 데이터 조회 🎯 

### SQL을 사용하여 Vegetable Details 테이블 조회 

이 챌린지 실습에서는 단계별 세부사항 대신, 여러 목표를 달성하기 위한 일반적인 지침을 제공합니다.

1. 데이터를 조회합니다.
1. "Spinach"에 해당되는 레코드를 분리합니다.
1. 데이터 세트에서 두 번 나타나는 plant name을 확인합니다.
1. 이를 제거할 방법을 찾아봅니다.
1. 데이터를 다시 확인합니다.

💡 **팁**: 아래 템플릿 쿼리에서 해시 문자(`#`)를 **수정**하세요. 해시 문자가 있는 곳에는 입력이 필요합니다. 

💡 **팁**: 진행 중 막혔다면, 각 템플릿 SQL 구문 밑에 있는 단서 셀의 오른쪽 상단 플레이 버튼 바로 옆에 있는 **View display options** 선택기에서 **Code and results**를 클릭하여 **확장**합니다. 먼저 스스로 해결해보세요. :grinning:

### 1. 데이터 조회 🎯 

In [ ]:
-- modify the following code and run to view all the table data after first setting your context (database and schema)
USE DATABASE {{user}}_GARDEN_PLANTS;
USE SCHEMA veggies;

SELECT *
FROM VEGETABLE_details;

In [ ]:
/*
-- you could also hard code the following USE statement for your animal user instead of using the variable substitution
USE SCHEMA {{user}}_garden_plants.veggies;

SELECT *
FROM vegetable_details;
*/

### 2. **plant_name**이 **Spinach**인 모든 행을 반환하는 SQL 쿼리 작성 🎯 

[`UPPER()`](https://docs.snowflake.com/ko/sql-reference/functions/upper) 함수를 사용하겠습니다. 이는 `spinach`, `Spinach` 또는 다른 변형 형태로 텍스트 문자열을 작성할 필요가 없음을 의미합니다. 입력값과 컬럼에 저장된 값이 모두 `SPINACH`로 변환됩니다.

💡 **힌트**: Spinach 행이 두 개나 있습니다!

In [ ]:
SELECT *
FROM VEGETABLE_DETAILS
WHERE UPPER(plant_name) = UPPER('SPINACH');

In [ ]:
/*
SELECT *
FROM vegetable_details
WHERE UPPER(plant_name) = UPPER('Spinach');
*/

### 3. 하나의 행은 얕은 뿌리를 나타내는 "S", 다른 행은 깊은 뿌리를 나타내는 "D"를 가집니다 🎯 

깊은 뿌리를 나타내는 Spinach 행을 제거해야 합니다. 먼저 "D" 행을 분리해 봅시다. 

In [ ]:
SELECT *
FROM vegetable_details
WHERE UPPER(plant_name) = UPPER('SPINACH')
AND root_depth_code = 'D';

In [ ]:
/*
SELECT *
FROM vegetable_details
WHERE UPPER(plant_name) = UPPER('Spinach')
AND root_depth_code = 'D';
*/

### 4. ROOT_DEPTH_CODE 컬럼에서 "D"를 가진 Spinach 행만 제거합니다. 🎯 

In [ ]:
DELETE
FROM vegetable_details
WHERE UPPER(plant_name) = UPPER('SPINACH')
AND root_depth_code = 'D';

In [ ]:
/*
DELETE
FROM vegetable_details
WHERE UPPER(plant_name) = UPPER('Spinach')
AND root_depth_code = 'D';
*/

### 5. 모든 데이터를 다시 확인하고, 중복된 Vegetable 이름이 없는지 확인합니다. 🎯 

In [ ]:
SELECT *
FROM VEGETABLE_DETAILS;

In [ ]:
/*
SELECT *
FROM vegetable_details;
*/

### File Format 오브젝트에 대한 참고 📓

이 실습에서 Snowsight의 Load Data Wizard를 사용하여 Snowflake가 파일에서 데이터를 읽고 테이블에 삽입하도록 가이드를 제공했습니다. 보셨다시피, 대화 상자에는 소스 파일에 있는 데이터의 "형태"를 Snowflake가 이해하도록 돕고, 파일 데이터를 어떻게 해석하고 처리해야 하는지 지정할 수 있는 옵션이 포함되어 있습니다. 여기에는 유형(예: CSV), 필드 구분자, 건너뛰어야 할 헤더 라인의 유무 등이 포함됩니다.

이러한 지침은 [FILE FORMAT](https://docs.snowflake.com/ko/sql-reference/sql/create-file-format)이라는 Snowflake 오브젝트로 묶을 수 있으며, 데이터 로드 대화 상자나 [`COPY INTO`](https://docs.snowflake.com/ko/sql-reference/sql/copy-into-table) 명령어를 통해 프로그래밍 방식으로 Snowflake에 데이터를 로드할 때 참조할 수 있습니다. 이를 통해 동일한 사양으로 파일을 반복 로드할 때 지침을 다시 입력하거나 코딩할 필요가 없어 시간을 절약할 수 있습니다.

## Cloning에 대한 간단한 소개 📓

Snowflake의 [zero-copy cloning](https://docs.snowflake.com/ko/user-guide/object-clone) 기능은 테이블, 스키마, 데이터베이스(및 다른 오브젝트)의 "스냅샷"을 빠르게 찍고 스토리지를 공유하는 해당 오브젝트의 파생 복사본을 생성하는 편리한 방법을 제공합니다. 이 과정은 기본 데이터의 물리적 복사를 수반하지 않고, 메타데이터만으로 이루어지는 작업입니다. 이로 인해 테이블, 스키마, 심지어 전체 데이터베이스를 클로닝하는 작업은 일반적으로 매우 빠릅니다. 

클론은 생성되는 순간부터 독립적인 오브젝트가 됩니다. 원본 오브젝트에 수행할 수 있는 모든 작업을 클론된 오브젝트에도 수행할 수 있습니다. 예를 들어, 클론된 테이블에 데이터를 쓰거나 `ALTER TABLE` 명령으로 파라미터를 변경하는 작업을 포함하는 모든 작업을 할 수 있습니다.

Cloning은 추가 비용없이 즉각적인 백업을 생성하는 데 매우 유용하며(클론된 오브젝트에 변경이 가해질 때까지), Snowflake에서 Dev 및 Test/QA 환경의 신속한 프로비저닝이나 데이터 백업에 자주 사용됩니다.

### 테이블 클론하기 🥋

다음 단계에서는 **(animal)_DB** 데이터베이스에 **CLONED_OBJECTS**라는 새로운 스키마를 생성합니다. 그런 다음 기존 **VEGETABLE_DETAILS** 테이블을 이 새로운 스키마로 **VEGETABLE_DETAILS_CLONE** 라는 이름으로 `CLONE`합니다. 

다음 코드를 실행하여 이 작업을 수행하고, 이 작업이 얼마나 빠르게 이루어지는지 확인해 보세요. Snowflake에서 Cloning은 테이블 데이터를 복사하지 않고 데이터에 대한 메타데이터(데이터에 대한 데이터)만 복사한다는 점을 기억하세요.

In [ ]:
CREATE SCHEMA IF NOT EXISTS {{user}}_db.cloned_objects;

USE SCHEMA {{user}}_db.cloned_objects;

CREATE TABLE IF NOT EXISTS vegetable_details_clone
    CLONE {{user}}_garden_plants.veggies.vegetable_details;

### 클론된 테이블의 세부정보 검토

`SHOW` 명령어를 사용하여 클론된 테이블과 원본 테이블의 세부 정보를 확인합니다. 이 두 테이블은 독립적은 오브젝트입니다. 그러나 두 테이블 모두 동일한 행 수와 바이트 수를 공유한다는 점을 참고하세요. 하지만 저장소 사용량은 원본 테이블에 대해서만 측정됩니다.

In [ ]:
SHOW TABLES LIKE 'vegetable_details%' IN ACCOUNT;

### 클론된 테이블 쿼리

클론된 테이블 **VEGETABLE_DETAILS_CLONE**을 원본 테이블 **VEGETABLE_DETAILS**애서 했던 것처럼 쿼리합니다. 이 데이터는 익숙하게 보일 것입니다! 클론된 오브젝트는 일반적인 Snowflake 테이블로, 생성 방식과 관계없이 모든 표준 테이블 작업을 지원합니다.

In [ ]:
SELECT * 
FROM vegetable_details_clone;

## Snowflake Marketplace 📓

### Snowflake Marketplace란 무엇일까요?

Snowflake Marketplace는 Snowflake 계정 내에서 액세스할 수 있는 통합된 포탈입니다. Snowflake Marketplace를 통해 써드 파티 데이터 및 서비스를 탐색하고 액세스하며, Snowflake Data Cloud를 통해 자신의 데이터 제품을 마케팅할 수 있습니다. 소비자로서 Snowflake Marketplace에서 제공하는 데이터를 사용하여 다음과 같은 작업을 수행할 수 있습니다:

- 연구, 예측, 머신러닝을 위한 과거 데이터 탐색
- 현재 날씨 및 교통 상황과 같은 최신 스트리밍 데이터 액세스
- 구독자 및 대상 청중을 이해하기 위한 특화된 식별 데이터 활용
- 예상치 못한 데이터 소스에서 새로운 인사이트 얻기

### Snowflake에서 테이블에 데이터를 로드하는 것보다 더 쉬운 것이 있을까요? 맞습니다, 데이터를 로드하지 않는 것입나다!

Snowflake Marketplace는 비즈니스에 유의미한 데이터와 데이터 제품을 찾기 위해 다양한 카테고리를 검색할 수 있는 기능을 제공합니다. 이는 점점 더 많아지고 있는 전 세계의 데이터 공급자 풀에서 데이터를 선택하여, 버튼 한번 만으로 계정 내에서 이를 활용할 수 있게 합니다. 수백 개의 목록이 제공되며, 이 중에는 **Free/Unlimited Access** 옵션도 포함되어 있었습니다. Marketplace 거래를 수행할 수 있는 권한이 있다면, 공유 데이터가 몇 초 내로 계정에서 사용할 수 있도록 할 수 있고, 새로운 데이터베이스로 나타납니다. 이 데이터는 그곳에서 분석 및 활용할 수 있으며, 심지어 자신의 데이터와 결합하여 새로운 데이터 제품을 증대, 향상 및 생성할 수도 있습니다. 데이터 로드는 전혀 필요하지 않습니다!

--- 

![Snowflake Marketplace (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_marketplace.png)

### Snowflake Marketplace 탐색 🥋 

1. 왼쪽 사이드 메뉴에서 **Data Products** > **Marketplace**를 선택합니다.
1. Snowflake Marketplace 검색창에서 "books"를 입력하고 엔터를 누르세요.
1. 나타나는 하위 메뉴에서 **Pricing** > **Free**를 선택하세요.
1. **AI Training Dataset from Goodreads Books** 목록을 찾으세요.
1. 링크를 클릭하여 데이터 세트에 대한 세부정보를 검토하세요. 여기에는 데이터 딕셔너리 사양과 사용 예시(SQL 쿼리)가 포함되어 있습니다.

💡 **팁**: Snowflake Marketplace Listing(목록)을 사용하려면 계정에 특별 권한이 필요합니다. 일반적으로 이는 Snowflake에서 고급 권한 역할을 가진 데이터 관리자 또는 시스템 관리자가 처리합니다.

## Data Exchange를 활용한 프라이빗 공유 📓

[Data Exchange](https://docs.snowflake.com/ko/user-guide/data-exchange)는 초대받은 특정 멤버 그룹과 데이터를 안전하게 협업할 수 있는 데이터 허브를 제공합니다. 공급자로서 데이터를 게시하면, Exchange에 참여하는 소비자들이 해당 데이터를 탐색하고 사용할 수 있습니다.

Data Exchange를 통해 Data Exchange에 참여하는 회사 내부의 부서나 회사 외부의 벤더, 공급업체, 파트너와 같은 일관된 비즈니스 파트너 그룹에 데이터를 쉽게 제공할 수 있습니다. 조직 내부 및 외부의 다양한 소비자와 데이터를 공유하고자 하는 경우, 특정 소비자를 대상으로 하거나 Snowflake Marketplace에서 공개적으로 제공되는 Listing(목록)을 활용할 수도 있습니다.

### 프라이빗 공유 사용 🥋 

"공개" Snowflake Marketplace 외에도, 데이터와 데이터 제품을 선별된 Snowflake 계정이나 계정 내 사용자와 공유할 수 있습니다. 이 기능을 **Data Exchange**라 합니다.

1. 왼쪽 사이드 메뉴 메인에서, Data Products > Private Sharing를 선택합니다.
1. 계정에 공유된 목록(Listing)을 검토하세요. 이중 많은 데이터가 Snowflake Education Services에서 제공되었습니다.
1. **Alpine Peaks Publishing** 목록을 클릭하고 세부정보를 검토합니다.

우리의 Snowflake 계정에서 이 데이터 세트를 기반으로 SQL 쿼리를 작성하여 우리의 관심 주제인 **gardening**에 대한 상위 도서 추천을 찾아볼 것입니다. 배울 내용은 항상 더 많습니다!!

💡 **참고**: Snowflake Education Services는 이미 프라이빗 Exchange 데이터 셋을 우리의 계정에 추가했습니다. 파란색 **GET** 버튼이나 **Get Access**링크를 클릭할 필요가 **없습니다**.

--- 

![Snowflake Data Exchange (image)](https://edu-cdev-images.s3.us-west-2.amazonaws.com/ob/ob_pde_alpine_peaks_listing_1.png)

- 이 공유 데이터를 현재 역할에서 접근 가능하게 하려면, 아래 SQL 구문을 실행하여 적절한 권한을 부여합니다.

- 오브젝트 브라우저를 새로고침하면 `ALPINE_PEAKS_PUBLISHING` 데이터베이스가 나타나는 것을 볼 수 있습니다.

In [ ]:
CALL common_db.resources.access_alpine_peaks_publishing_data('{{user}}_learner_rl');

### 프라이빗 Data Exchange 데이터 쿼리 🥋

Data Exchange의 **Alpine Peaks Publishing** 목록에는 가음과 같은 설명이 나타납니다:

- _Alpine Peaks Publishing, a fictional book publisher, has unveiled its upcoming 2025 catalog to Snowflake Education. The collection features various titles across multiple genres, appealing to a wide audience. Showcasing the work of their acclaimed (but fictional) authors, this lineup promises to captivate readers with something for everyone._

- (번역)_Alpine Peaks Publishing은 가상의 도서 출판사로, Snowflake Education에 2025년 예정된 카탈로그를 공개했습니다. 이 컬렉션은 다양한 장르의 여러 제목을 포함하고 있어 폭넓은 독자층에게 매력적으로 다가갑니다. 이 가상의 저명한 작가들의 작품을 선보이는 이번 라인업은 모든 독자를 사로잡을 수 있는 내용을 약속합니다._

다음 SQL 셀에 있는 쿼리를 실행하여 이 데이터 세트를 탐색해 보세요!

In [ ]:
-- how many book reviews in this data set?
SELECT COUNT(*) 
FROM alpine_peaks_publishing.books.forthcoming_releases;

In [ ]:
-- take a look at a subset of this data 
SELECT * 
FROM alpine_peaks_publishing.books.forthcoming_releases
LIMIT 10;

In [ ]:
-- determine the number of unique categories in this data set 
SELECT DISTINCT category 
FROM alpine_peaks_publishing.books.forthcoming_releases
ORDER BY 1;

In [ ]:
-- produce a count of the total number of books per category 
SELECT category, COUNT(*) 
FROM alpine_peaks_publishing.books.forthcoming_releases
GROUP BY category 
ORDER BY 2 DESC;

In [ ]:
-- find books with 'garden' in their name
SELECT *
FROM alpine_peaks_publishing.books.forthcoming_releases
WHERE LOWER(title) LIKE '%garden%';

In [ ]:
-- list the books from our favorite category 'gardening' and order them by the date they will become available in 2025
SELECT *
FROM alpine_peaks_publishing.books.forthcoming_releases
WHERE CONTAINS(LOWER(category),'garden')
ORDER BY release_date;


## Lesson 6 Wrap Up 🏁 

### Lesson 6을 완료할 준비가 되셨나요? 🏁 

- **ROOT_DEPTH** 테이블은 3개의 행을 갖고 있나요?

- **VEGETABLE_DETAILS** 테이블은 41개의 행을 갖고 있나요?

- 두 테이블 모두 **(animal)_GARDEN_PLANTS** 데이터베이스의 **VEGGIES** 스키마에 있나요? 

이 모든 질문에 "예"라고 답한다면, 이 수업을 완료한 것입니다. 그렇지 않은 경우 다시 돌아가서 잘못된 부분을 수정해야 합니다!

## 지식 테스트 🔎

아래 Python 셀을 실행하여 Streamlit 기반의 위젯을 표시하고 Snowflake 특징과 기능에 대한 질문에 답하세요. 지금은 이 코드가 무엇을 하는지 이해할 필요가 없습니다. 그냥 코드를 실행하세요.

질문에 올바르게 답해야 다음 섹션으로 진행할 수 있습니다.

In [ ]:
st.divider()
question = "File Format이란 무엇입니까?"
options = ["아래 보기 중 고르세요...",
           "A) 데이터가 도착할 때 Snowflake에 데이터 구조를 알리는 방법입니다.", 
           "B) 테이블의 여백과 글꼴을 설정하는 방법입니다.", 
           "C) 웹 페이지의 배경색을 선택하는 방법입니다."]

user_answer = st.radio(question, options, index=0)
if user_answer:
    if user_answer == "아래 보기 중 고르세요...":
        ''
    else:
        answer = '2cfc0e9ef4c5d4ce80db59f462539062'
        response = session.sql(f"call common_db.resources.quiz_temp('{answer}', '{user_answer}', 'False')").collect()
        if response:
            value = response[0]['QUIZ_TEMP']
        st.write(value)

## 다음 단계

실습 단계를 완료하고 **지식 테스트** 질문에 올바르게 답했다면, Snowflake 강사의 안내에 따라 다음 노트북으로 진행하세요.